# YOLOv8 Number Plate Detection Training

This notebook trains a YOLOv8 model for number plate detection using a custom dataset.

**Hardware:** Intel i9 14th Gen, RTX 4060 8GB GPU

## 1. Install Required Packages

In [1]:
# Install PyTorch with CUDA 12.1 support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Install other packages from PyPI
!pip install ultralytics scikit-learn pyyaml

print("✓ All packages installed successfully!")

Looking in indexes: https://download.pytorch.org/whl/cu121
✓ All packages installed successfully!


## 2. Import Libraries and Check GPU

In [2]:
import torch
from ultralytics import YOLO
import os
from pathlib import Path

# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\cindr\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PyTorch version: 2.5.1+cu121
CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU
GPU Memory: 8.00 GB


## 3. Prepare Dataset Structure

In [3]:
# Check dataset structure
dataset_path = Path('dataset')
images_path = dataset_path / 'images'
labels_path = dataset_path / 'labels'

print(f"Dataset path exists: {dataset_path.exists()}")
print(f"Images path exists: {images_path.exists()}")
print(f"Labels path exists: {labels_path.exists()}")

# Count files
if images_path.exists():
    image_files = list(images_path.glob('*.*'))
    print(f"Number of images: {len(image_files)}")
    
if labels_path.exists():
    label_files = list(labels_path.glob('*.txt'))
    print(f"Number of label files: {len(label_files)}")
    
    # Sample a label file to check format
    if label_files:
        sample_label = label_files[0]
        print(f"\nSample label file: {sample_label.name}")
        with open(sample_label, 'r') as f:
            print(f"Content:\n{f.read()}")

Dataset path exists: True
Images path exists: True
Labels path exists: True
Number of images: 8078
Number of label files: 8078

Sample label file: 2024-03-08_1002_0.txt
Content:
0 0.5086771916963694 0.41577595891179264 0.2718135924807278 0.06956906705289273



## 4. Split Dataset into Train/Val Sets

In [4]:
import shutil
from sklearn.model_selection import train_test_split

# Create train/val directory structure
train_images_dir = dataset_path / 'train' / 'images'
train_labels_dir = dataset_path / 'train' / 'labels'
val_images_dir = dataset_path / 'val' / 'images'
val_labels_dir = dataset_path / 'val' / 'labels'

for dir_path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

# Get all label files
label_files = list(labels_path.glob('*.txt'))

# Split 80/20 for train/val
train_labels, val_labels = train_test_split(label_files, test_size=0.2, random_state=42)

print(f"Training samples: {len(train_labels)}")
print(f"Validation samples: {len(val_labels)}")

# Copy files to train directory
for label_file in train_labels:
    # Copy label
    shutil.copy2(label_file, train_labels_dir / label_file.name)
    
    # Find and copy corresponding image
    base_name = label_file.stem
    for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:
        img_file = images_path / f"{base_name}{ext}"
        if img_file.exists():
            shutil.copy2(img_file, train_images_dir / img_file.name)
            break

# Copy files to val directory
for label_file in val_labels:
    # Copy label
    shutil.copy2(label_file, val_labels_dir / label_file.name)
    
    # Find and copy corresponding image
    base_name = label_file.stem
    for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:
        img_file = images_path / f"{base_name}{ext}"
        if img_file.exists():
            shutil.copy2(img_file, val_images_dir / img_file.name)
            break

print("\nDataset split completed!")
print(f"Train images: {len(list(train_images_dir.glob('*.*')))}")
print(f"Val images: {len(list(val_images_dir.glob('*.*')))}")

Training samples: 6462
Validation samples: 1616

Dataset split completed!
Train images: 6462
Val images: 1616


## 5. Create data.yaml Configuration File

In [5]:
import yaml

# Create data.yaml for YOLO
data_yaml = {
    'path': str(dataset_path.absolute()),
    'train': 'train/images',
    'val': 'val/images',
    'nc': 1,  # number of classes (number plate)
    'names': ['number_plate']
}

yaml_path = 'data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print("data.yaml created successfully!")
print("\nContents:")
with open(yaml_path, 'r') as f:
    print(f.read())

data.yaml created successfully!

Contents:
names:
- number_plate
nc: 1
path: c:\Users\cindr\Desktop\project\dataset
train: train/images
val: val/images



## 6. Initialize YOLOv8 Model

In [6]:
# Load YOLOv8 model (yolov8n = nano, fastest; yolov8s = small; yolov8m = medium)
# Using nano for 8GB GPU
model = YOLO('yolov8n.pt')  # pretrained model
print("Model loaded successfully!")

Model loaded successfully!


## 7. Train the Model

Training parameters optimized for RTX 4060 8GB GPU:
- **Batch size**: 16 (adjust if GPU runs out of memory)
- **Image size**: 640 (standard YOLO size)
- **Epochs**: 100 (adjust based on results)
- **Device**: GPU (cuda:0)

In [7]:
# Train the model
results = model.train(
    data='data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,  # Adjust if OOM (out of memory) - try 8 or 4
    device=0,  # Use GPU 0
    workers=8,  # Parallel data loading
    project='runs/detect',
    name='numberplate_detection',
    patience=20,  # Early stopping patience
    save=True,
    plots=True,
    amp=True,  # Automatic Mixed Precision for faster training
    cache=False,  # Set to 'ram' if you have enough RAM (32GB+)
)

print("\nTraining completed!")

Ultralytics 8.3.237  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=numberplate_detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, p

## 8. Validate the Model

In [8]:
# Validate the trained model
metrics = model.val()

print(f"\nValidation Results:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

Ultralytics 8.3.237  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 582.1251.7 MB/s, size: 70.5 KB)
val: Scanning C:\Users\cindr\Desktop\project\dataset\val\labels.cache... 1616 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1616/1616 1.6Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 101/101 10.5it/s 9.6s0.1s
                   all       1616       1740      0.974      0.966      0.991      0.946
Speed: 0.8ms preprocess, 2.1ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to C:\Users\cindr\Desktop\project\runs\detect\val

Validation Results:
mAP50: 0.9908
mAP50-95: 0.9458
Precision: 0.9740
Recall: 0.9661


## 9. Test Predictions on Sample Images

In [9]:
# Load best trained model
best_model = YOLO('runs/detect/numberplate_detection/weights/best.pt')

# Get a sample validation image
val_images = list(val_images_dir.glob('*.*'))
if val_images:
    sample_image = val_images[0]
    
    # Run prediction
    results = best_model.predict(
        source=sample_image,
        conf=0.25,  # Confidence threshold
        device=0,
        save=True,
        project='runs/detect',
        name='test_predictions'
    )
    
    print(f"Prediction saved to: runs/detect/test_predictions")
    print(f"Number of detections: {len(results[0].boxes)}")
else:
    print("No validation images found!")


image 1/1 c:\Users\cindr\Desktop\project\dataset\val\images\2024-03-08_1017_7.jpg: 480x640 2 number_plates, 45.7ms
Speed: 3.4ms preprocess, 45.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to C:\Users\cindr\Desktop\project\runs\detect\test_predictions
Prediction saved to: runs/detect/test_predictions
Number of detections: 2


## 10. Export Model for Inference

Export the trained model to different formats for deployment.

In [10]:
# Export model to ONNX format (optional)
# best_model.export(format='onnx')

print("\nTraining Results Location:")
print(f"Best weights: runs/detect/numberplate_detection/weights/best.pt")
print(f"Last weights: runs/detect/numberplate_detection/weights/last.pt")
print(f"Training plots: runs/detect/numberplate_detection/")

print("\n✓ Training pipeline completed successfully!")


Training Results Location:
Best weights: runs/detect/numberplate_detection/weights/best.pt
Last weights: runs/detect/numberplate_detection/weights/last.pt
Training plots: runs/detect/numberplate_detection/

✓ Training pipeline completed successfully!
